In [1]:
import numpy as np
import pandas as pd
#import litstudy as lst
import nltk
#import Lbl2Vec as vec
import pandas as pd
import numpy as np#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer#for word embedding
import gensim
from gensim.models import Word2Vec

In [2]:
master = pd.read_csv("captive_wild_animals.csv")
master = master[["Title", "Authors", "Abstract", "Published Year", "Accepted"]]
master.isna().sum()

Title               0
Authors            22
Abstract          107
Published Year      1
Accepted            0
dtype: int64

In [3]:
master = master.dropna(ignore_index=True)
master.isna().sum()

Title             0
Authors           0
Abstract          0
Published Year    0
Accepted          0
dtype: int64

In [4]:
master

,Title,Authors,Abstract,Published Year,Accepted
0,"One Health, emerging infectious diseases and w...",Cunningham AA; Daszak P; Wood JLN,"Infectious diseases affect people, domestic an...",2017,0
1,COVID-19-Zoonosis or Emerging Infectious Disease?,Haider N; Rothman-Ostrow P; Osman AY; Arruda L...,The World Health Organization defines a zoonos...,2020,0
2,The role of wildlife in emerging and re-emergi...,Bengis RG; Leighton FA; Fischer JR; Artois M; ...,There are huge numbers of wild animals distrib...,2004,0
3,Global hotspots and correlates of emerging zoo...,Allen T; Murray KA; Zambrana-Torrelio C; Morse...,Zoonoses originating from wildlife represent a...,2017,0
4,"Brucellosis in humans--etiology, diagnostics, ...",Galińska EM; Zagórski J,Brucellosis in humans is a zoonosis of greatly...,2013,0
...,...,...,...,...,...
5529,Occurrence of ten protozoan enteric pathogens ...,"Menu, E.; Davoust, B.; Mediannikov, O.; Akiana...",Non-human primate populations act as potential...,2021,1
5530,Molecular and serological detection of Toxopla...,Hsu ChinWei; Wang PaoJung; Huang PeiYun; Lien ...,Toxoplasmosis caused by Toxoplasma gondii affe...,2022,1
5531,Sequence analysis of novel Staphylococcus aure...,"Monecke, S.; Roberts, M. C.; Braun, S. D.; Die...",Staphylococcus aureus is a widespread and comm...,2022,1
5532,Intestinal parasites in captive wild animals a...,Qiu HongYu; Yang HuiNing; Sun XiaoJing; Jin Ch...,Objective: In order to investigate the prevale...,2022,1


In [27]:
### remove stopwords and punctuation.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as tokenize

words = tokenize(master["abstract"][1])
stop_words = set(stopwords.words("english"))
extra = {",", ".", "(", ")", ";", ":", "=", "[0-9]"}
filtered_list = []
for word in words:
    if word.casefold() not in stop_words and word.casefold() not in extra:
         filtered_list.append(word)

In [28]:
### stemming
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")
stemmed = [stemmer.stem(word) for word in filtered_list]

In [33]:
abstracts = master["abstract"]

In [41]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as tokenize
from nltk.stem import SnowballStemmer

nlp_dict = {}
all_words = []
row = int(0)
for abstract in abstracts:
    words = tokenize(abstract)
    stop_words = set(stopwords.words("english"))
    extra = {",", ".", "(", ")", ";", ":", "="}
    filtered_list = []
    for word in words:
        if word.casefold() not in stop_words and word.casefold() not in extra:
             filtered_list.append(word)
    
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in filtered_list]
    nlp_dict[row] = stemmed
    all_words.append(stemmed)
    row += 1

In [5]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text
 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN# Tokenize the sentence

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\harrg\AppData\Local\Temp\ipykernel_20876\925071319.py:7: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)


In [6]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))


master['abstract_clean'] = master['Abstract'].apply(lambda x: finalpreprocess(x))

In [7]:
master["abstract_clean"]

0       infectious disease affect people domestic anim...
1       world health organization defines zoonosis inf...
2       huge number wild animal distribute throughout ...
3       zoonosis originate wildlife represent signific...
4       brucellosis human zoonosis greatly varied clin...
                              ...                        
5529    non human primate population act potential res...
5530    toxoplasmosis cause toxoplasma gondii affect c...
5531    staphylococcus aureus widespread common opport...
5532    objective order investigate prevalence intesti...
5533    west nile virus wnv usutu virus usuv zoonotic ...
Name: abstract_clean, Length: 5534, dtype: object

In [40]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(master["abstract_clean"],master['Accepted'],test_size=0.9, shuffle=True)#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [41]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [20]:
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
        
    def fit(self, X, y):
        return self
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

master['abstract_clean_tok']=[nltk.word_tokenize(i) for i in master['abstract_clean']]
model = Word2Vec(master['abstract_clean_tok'],min_count=1, vector_size=100)     
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))
modelw = MeanEmbeddingVectorizer(w2v)# converting text to numerical data using Word2Vec
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_test_tok)

In [42]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression
lr_tfidf=LogisticRegression(solver = 'liblinear', C=0.1, penalty = 'l2', class_weight={0: 1., 1: 100.})
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  #Predict y value for test dataset
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4724
           1       0.05      1.00      0.10       257

    accuracy                           0.05      4981
   macro avg       0.03      0.50      0.05      4981
weighted avg       0.00      0.05      0.01      4981

Confusion Matrix:
[[   0 4724]
 [   0  257]]
AUC: 0.7965093388508717


C:\Users\harrg\AppData\Local\anaconda3\envs\litsearch\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harrg\AppData\Local\anaconda3\envs\litsearch\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harrg\AppData\Local\anaconda3\envs\litsearch\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo